In [1]:
import os

import pandas as pd

In [2]:
protostomia = pd.concat(
    [
        pd.read_csv("../results_summary/datasets_Cnidaria_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/datasets_Ecdysozoa_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/datasets_Spiralia_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/psi_blast_Anthozoa_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/psi_blast_Bivalvia_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/psi_blast_Gastropoda_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/psi_blast_Nematoda_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/psi_blast_Platyhelminthes_cds_cassette.tsv", sep="\t"),
    ], ignore_index=True
)

# ищем дубликаты
# protostomia[protostomia["org_name"].duplicated(keep=False)]

# удаляем дубликаты
# -----------------
# 64 - Actinia_tenebrosa__XP_031555922.1
# 111 - Aplysia_californica__XP_005094816.1
# 88 - Crassostrea_virginica__XP_022333190.1
# 95 - Magallana_gigas__XP_065942772.1
# -----------------
indices_to_drop = [64, 111, 88, 95]
protostomia = protostomia.drop(indices_to_drop)

deuterostomia = pd.concat(
    [
        pd.read_csv("../results_summary/datasets_Actinopterygii_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/datasets_Amphibia_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/datasets_Sauropsida_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/psi_blast_Caudata_cds_cassette.tsv", sep="\t"),
        pd.read_csv("../results_summary/psi_blast_Gymnophiona_cds_cassette.tsv", sep="\t"),
    ], ignore_index=True
)

# ищем дубликаты
# deuterostomia[deuterostomia["org_name"].duplicated(keep=False)]

# удаляем дубликаты
# -----------------
# 172 - Pleurodeles_waltl__KAJ1108515.1
# 173 - Rhinatrema_bivittatum__XP_029448920.1
# -----------------
indices_to_drop = [172, 173]
deuterostomia = deuterostomia.drop(indices_to_drop)

# сохраняем итоговые таблицы
protostomia.to_csv("../results_summary/protostomia.tsv", index=False, sep="\t")
deuterostomia.to_csv("../results_summary/deuterostomia.tsv", index=False, sep="\t")

---